In [1]:
# 首先，我们暂时忽略通道（第三维）这一情况，看看如何处理二维图像数据和隐藏表示
# 稍后，我们将看到如何通过在图像边界周围填充零来保证有足够的空间移动卷积核，从而保持输出大小不变。
# 接下来，我们在corr2d函数中实现如上过程，该函数接受输入张量X和卷积核张量K，并返回输出张量Y

# torch.rand(size) # 均匀分布U(0, 1)
# torch.randn(size) # 高斯正太, random normal
# torch.normal(mean, std, size)
# torch.arange(start, end, step)
# torch.FloatTensor([]) 创建一个float32的张量

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

# correlation 2d 互相关运算-2d
def corr2d(X, K): #@save
    # 计算二维互相关运算
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1 )) # 输出矩阵大小
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j + w] * K).sum()
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], 
                  [3.0, 4.0, 5.0], 
                  [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], 
                  [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [4]:
# 卷积层对输入和卷积核权重进行互相关运算，并在添加标量偏置之后产生输出。 所以，卷积层中的两个被训练的参数是卷积核权重和标量偏置。 
# 就像我们之前随机初始化全连接层一样，在训练基于卷积层的模型时，我们也随机初始化卷积核权重。

In [5]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Paramater(torch.rand(kernel_size)) # 卷积核权重
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, X):
        return corr2d(x, self.weight) + self.bias

In [6]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [7]:
K = torch.tensor([[1.0, -1.0]])

In [8]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [9]:
# 现在我们将输入的二维图像转置，再进行如上的互相关运算

In [10]:
corr2d(X.t(), K) # 垂直边缘消失了，不出所料，只能检测垂直边缘

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [11]:
# 现在让我们看看是否可以通过仅查看“输入-输出”对来学习由X生成Y的卷积核。 
# 我们先构造一个卷积层，并将其卷积核初始化为随机张量。
# 接下来，在每次迭代中，我们比较Y与卷积层输出的平方误差，然后计算梯度来更新卷积核。
# 为了简单起见，我们在此使用内置的二维卷积层，并忽略偏置

In [12]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias = False) # in_channels, out_channels, size

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(20):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 6.183
epoch 4, loss 1.969
epoch 6, loss 0.712
epoch 8, loss 0.276
epoch 10, loss 0.110
epoch 12, loss 0.045
epoch 14, loss 0.018
epoch 16, loss 0.007
epoch 18, loss 0.003
epoch 20, loss 0.001


In [13]:
# 我们这里是在让这个conv2D 学习我们先前算出来的Y

In [14]:
conv2d.weight.view(1, -1), conv2d.weight.data.reshape(1, -1)

(tensor([[ 0.9963, -1.0036]], grad_fn=<ViewBackward0>),
 tensor([[ 0.9963, -1.0036]]))

##### 接下来介绍填充和步幅

In [18]:
# 如果我们图像为 nh,nw ; 卷积核 kh,kw ; 我们填充 ph,pw(指一半在上面一半下面，一共ph)
# 那一般来说我们的输出形状就会变成 (nh + ph - kh + 1) * (nw + pw - kw + 1)
# 一般来说 我们设置 ph = kh - 1 ; pw = kw - 1 
# k选择奇数的好处是，保持空间维度的同时，我们可以在顶部和底部填充相同数量的行，在左侧和右侧填充相同数量的列。

In [29]:
# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数

def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape) # 元组的链接操作
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道s
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size = 3, padding = 1)
X = torch.rand(size = (8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [30]:
# 而当有步幅 sh,sw时，输出形状就变成 [(nh + ph - kh + sh) / sh ] * [(nw + pw - kw + sw) / sw]

conv2d = nn.Conv2d(1, 1, kernel_size = 3, padding = 1, stride = 2)
comp_conv2d(conv2d, X).shape
# 长: 8 + 2 - 3 + 2 / 2 = 4

torch.Size([4, 4])